In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/dolly

Mounted at /content/drive
/content/drive/MyDrive/dolly


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy  #ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
ERROR: pip's depende

In [ ]:
# from accelerate import FullyShardedDataParallelPlugin, Accelerator
# from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

# fsdp_plugin = FullyShardedDataParallelPlugin(
#     state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
#     optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
# )

# accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)

train_dataset = raw_datasets.get("train")
eval_dataset = raw_datasets.get("validation")
test_dataset = raw_datasets.get("test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


In [4]:
def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


After fine-tuning

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "databricks/dolly-v2-3b"
# base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    # use_auth_token=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "dolly_v2_3b-orkg-finetune")

In [8]:
import json

def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def load_json(file__name):
    try:
        data_file = open(file__name, "r", encoding='utf-8')
        file_data = json.loads(data_file.read())
        data_file.close()
        return file_data
    except FileNotFoundError:
        return None


def check_result(res):
    st = "output (Sparql query):"
    if st in res:
        loc = res.index(st)+len(st)
        res = res[loc:]
    return res


def main():
    data = load_json("ft_dolly.json")
    if data is None:
        questions = [x['question']['string'] for x in test_dataset]
        sparql = [clean(x["query"]["sparql"]) for x in raw_datasets.get("test")]
        gs = []
    else:
        questions = data["questions"]
        sparql = data["sparql"]
        # gs = [check_result(clean(x)).strip() for x in data["generated_sparql"]]
        gs = data["generated_sparql"]

    for q in questions[len(gs):]:
        eval_prompt = """Return a Sparql query for ORKG for the English text given in input.
        input (English text): {}
        output (Sparql query):
        """.format(q)
        model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")
        ft_model.eval()
        with torch.no_grad():
            res = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True)
        res = clean(res)
        res = check_result(res).strip()
        print(len(gs), res)
        gs.append(res)
        result = {"questions": questions, "sparql": sparql, "generated_sparql": gs, "suggestions": []}
        save_json("ft_dolly.json", result)

In [9]:
main()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


0 SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Accuracy") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Story Cloze Test") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } ORDER BY DESC( ?value) LIMIT 1 } } { SELECT ?model ?model_lbl WHERE { ?model rdfs:label ?model_lbl. } 	 } 	 { SELECT ?model ?model_lbl WHERE { ?model orkgp:HAS_SOURCE_CODE ?code; orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } 	 } 	 { SELECT ?model ?model_lbl WHERE { ?model orkgp:HAS_SOURCE_CODE ?code; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } 	 { SELECT ?model ?model_lbl WHERE { ?model orkgp:HAS_MODEL ?model; orkgp:HAS_MODEL_PARAMETER ?param. ?param orkgp:H

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1 SELECT DISTINCT ?paper ?paper_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Penn Treebank (Word Level)") ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark. ?paper orkgp:P31 ?cont; rdfs:label ?paper_lbl. } ORDER BY ?cont orkgp:HAS_VALUE ?value; rdfs:label ?cont_lbl. OFFSET ?offset orkgp:HAS_OFFSET ?offset_value. LIMIT ?limit orkgp:HAS_LIMIT ?limit_value. ( ?cont orkgp:HAS_VALUE ?value; rdfs:label ?cont_lbl) ORKG ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_DATASET ?dataset. rya:label ?dataset_lbl rdfs:label ?cont_lbl. cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_DATASET ?dataset. rya:label ?paper_lbl rdfs:label ?cont_lbl. paper orkgp:P31 ?cont; orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_DATASET ?dataset. rya:label ?paper_lbl:label rdfs:label ?cont_lbl:label. ( ?cont orkgp:HAS_VALUE ?value; orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_DATASET ?dataset) rya:label ?cont_lbl:label rdfs:label ?value. ( ?value orkgp:H

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

509 SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Score") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Atari 2600 Q*Bert") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } ORDER BY DESC( ?value) LIMIT 1 } } \n		SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Atari 2600 Q*Bert") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric_lbl. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } \n		SELECT ?model ?model_lbl WHERE { ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric_lbl. ?cont orkgp:HAS_BENCHMARK

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


510 SELECT DISTINCT ?metric ?metric_lbl (MAX( ?value) AS ?score) WHERE { { SELECT ?metric ?metric_lbl ?value WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Words in Context") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value. OPTIONAL { ?eval orkgp:HAS_METRIC ?metric. ?metric rdfs:label ?metric_lbl. } ?cont orkgp:HAS_BENCHMARK ?benchmark. OPTIONAL { ?cont orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } } ORDER BY DESC( ?value) } } GROUP BY ?metric ?metric_lbl


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


511 SELECT DISTINCT ?dataset ?dataset_lbl WHERE { ?problem a orkgc:Problem; rdfs:label ?problem_lbl. FILTER (str( ?problem_lbl) = "Common Sense Reasoning") ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. ?benchmark orkgp:HAS_DATASET ?dataset. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. } ORDER BY DESC(COUNT( ?dataset_lbl)) LIMIT 1 SELECT ?dataset ?dataset_lbl WHERE { ?cont orkgp:HAS_BENCHMARK ?benchmark; rdfs:label ?cont_lbl. } ?problem rdfs:label ?problem_lbl; orkgp:HAS_PROPERTY ?property. ?property rdfs:label ?property_lbl. } input (English text): List the datasets benchmarked under the Common Sense Reasoning research problem ? output (Sparql query): SELECT DISTINCT ?dataset ?dataset_lbl WHERE { ?cont orkgp:HAS_BENCHMARK ?benchmark; rdfs:label ?cont_lbl. ?cont orkgp:HAS_SOURCE_CODE ?code. ?code rdfs:label ?code_lbl. } ORDER BY DESC(COUNT( ?dataset_lbl)) LIMIT 1 SELECT ?dataset ?dataset_lbl WHERE { ?cont orkgp:HAS_SOURCE_CODE ?code; rdfs:label ?cont_lbl. ?cont orkgp:H

In [ ]:
eval_prompt = """Return a Sparql query for ORKG for the English text given in input.
input (English text): Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?
output (Sparql query):
"""
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Return a Sparql query for ORKG for the English text given in input.
input (English text): Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?
output (Sparql query):
    SELECT DISTINCT ?model ?model_lbl WHERE { ?metric a orkgc:Metric; rdfs:label ?metric_lbl. FILTER (str( ?metric_lbl) = "Accuracy") { SELECT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Story Cloze Test") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value; orkgp:HAS_METRIC ?metric. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } ORDER BY DESC( ?value) LIMIT 1 } } 
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
